In [6]:
import numpy as np # For data manipulation

# train_location = r"/kaggle/input/fraud-detection/fraudTrain.csv"

train_location = r"C:\Users\Kunjal Thorat\Desktop\Card Fraud detection\fraudTrain.csv"
train_df = pd.read_csv(train_location)

# test_location = r"/kaggle/input/fraud-detection/fraudTest.csv"

test_location = r"C:\Users\Kunjal Thorat\Desktop\Card Fraud detection\fraudTest.csv"
test_df = pd.read_csv(test_location)

train_df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [7]:
train_df.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [8]:
test_df.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [9]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


def data_pre(X):
    # Columns to drop
    del_col=['merchant','first','last','street','zip','unix_time','Unnamed: 0','trans_num','cc_num']
    X.drop(columns=del_col, inplace=True)

    # Data Conversion
    X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time'])
    X['trans_date'] = X['trans_date_trans_time'].dt.strftime('%Y-%m-%d')
    X['trans_date'] = pd.to_datetime(X['trans_date'])
    X['dob'] = pd.to_datetime(X['dob'])

    # Calculate Age of each transaction
    X["age"] = (X["trans_date"] - X["dob"]).dt.days // 365
    X['trans_month'] = X['trans_date'].dt.month
    X['trans_year'] = X['trans_date'].dt.year

    # Convert 'gender' to binary values
    X['gender'] = X['gender'].apply(lambda x: 1 if x == 'M' else 0)
    X['gender'] = X['gender'].astype(int)

    # Calculate absolute differences
    X['lat_dis'] = abs(X['lat'] - X['merch_lat'])
    X['long_dis'] = abs(X['long'] - X['merch_long'])

    # One-hot encoding for 'category'
    X = pd.get_dummies(X, columns=['category'])
    X = X.drop(columns=['city','trans_date_trans_time','state','job','merch_lat','merch_long','lat','long','dob','trans_date'])
    return X

# Preprocess the training dataset
train_df_pre = data_pre(train_df.copy())

# Preprocess the testing dataset
test_df_pre = data_pre(test_df.copy())


In [10]:
train_df_pre=data_pre(train_df.copy())
train_df_pre.head()


,amt,gender,city_pop,is_fraud,age,trans_month,trans_year,lat_dis,long_dis,category_entertainment,...,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,4.97,0,3495,0,30,1,2019,0.067507,0.870215,0,...,0,0,0,0,1,0,0,0,0,0
1,107.23,0,149,0,40,1,2019,0.271247,0.024038,0,...,1,0,0,0,0,0,0,0,0,0
2,220.11,1,4154,0,56,1,2019,0.969904,0.107519,1,...,0,0,0,0,0,0,0,0,0,0
3,45.00,1,1939,0,52,1,2019,0.803731,0.447271,0,...,0,0,0,0,0,0,0,0,0,0
4,41.96,1,99,0,32,1,2019,0.254299,0.830441,0,...,0,0,0,0,0,1,0,0,0,0


In [11]:
test_df_pre=data_pre(test_df.copy())
test_df_pre.head()

,amt,gender,city_pop,is_fraud,age,trans_month,trans_year,lat_dis,long_dis,category_entertainment,...,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,2.86,1,333497,0,52,6,2020,0.020491,0.265214,0,...,0,0,0,0,0,0,1,0,0,0
1,29.84,0,302,0,30,6,2020,0.870202,0.475569,0,...,0,0,0,0,0,0,1,0,0,0
2,41.28,0,34496,0,49,6,2020,0.177090,0.659611,0,...,0,1,0,0,0,0,0,0,0,0
3,60.05,1,54767,0,32,6,2020,0.242698,0.063961,0,...,0,0,0,0,0,1,0,0,0,0
4,3.19,1,1126,0,65,6,2020,0.706248,0.867734,0,...,0,0,0,0,0,0,0,0,0,1


In [12]:
# Split the training data into features (x_train) and the target variable (y_train)
x_train = train_df_pre.drop('is_fraud', axis=1)
y_train = train_df_pre['is_fraud']

# Split the testing data into features (x_test) and the target variable (y_test)
x_test = test_df_pre.drop('is_fraud', axis=1)
y_test = test_df_pre['is_fraud']


In [14]:
from sklearn.preprocessing import StandardScaler
# Fit the StandardScaler on the training data
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [23]:
# Logistic Regression Model:
from sklearn.linear_model import LogisticRegression

# Create and fit the Logistic Regression model
logistic_regression = LogisticRegression()
logistic_regression.fit(x_train, y_train)

# Make predictions
y_pred_logistic = logistic_regression.predict(x_test)

# Calculate accuracy
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
print("Accuracy for Logistic Regression:", accuracy_logistic)

# Display the classification report
print("\nClassification Report for Logistic Regression:\n", classification_report(y_test, y_pred_logistic))

Accuracy for Logistic Regression: 0.9956164896287513

Classification Report for Logistic Regression:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.00      0.00      0.00      2145

    accuracy                           1.00    555719
   macro avg       0.50      0.50      0.50    555719
weighted avg       0.99      1.00      0.99    555719



In [44]:
# Create a new DataFrame to store the results for Logistic Regression
results_logistic = pd.DataFrame({
    'Card_Identifier': test_df['cc_num'],  # Assuming 'cc_num' is the card identifier
    'Logistic_Reg_Detection': y_pred_logistic,
    'Fraud_Status': ['Fraud' if detection == 1 else 'Legitimate' for detection in y_pred_logistic]
})

# Initialize a count for fraud detected
count_fraud_logistic = 0

# Iterate through the results for Logistic Regression and count the fraud detected
for index, row in results_logistic.iterrows():
    if row['Fraud_Status'] == 'Fraud':
        count_fraud_logistic += 1

# Print the count of fraud detected for Logistic Regression
print(f"Count of Fraud Detected for Logistic Regression: {count_fraud_logistic}")

Count of Fraud Detected for Logistic Regression: 291


In [25]:
#Decision Tree Model:
from sklearn.tree import DecisionTreeClassifier

# Create and fit the Decision Tree model
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train)

# Make predictions
y_pred_dt = decision_tree.predict(x_test)

# Calculate accuracy
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Accuracy for Decision Tree:", accuracy_dt)

# Display the classification report
print("\nClassification Report for Decision Tree:\n", classification_report(y_test, y_pred_dt))

Accuracy for Decision Tree: 0.9965504148679458

Classification Report for Decision Tree:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.55      0.60      0.58      2145

    accuracy                           1.00    555719
   macro avg       0.77      0.80      0.79    555719
weighted avg       1.00      1.00      1.00    555719



In [45]:
# Create a new DataFrame to store the results for Decision Tree
results_decision_tree = pd.DataFrame({
    'Card_Identifier': test_df['cc_num'],  # Assuming 'cc_num' is the card identifier
    'Decision_Tree_Detection': y_pred_dt,
    'Fraud_Status': ['Fraud' if detection == 1 else 'Legitimate' for detection in y_pred_dt]
})

# Initialize a count for fraud detected
count_fraud_decision_tree = 0

# Iterate through the results for Decision Tree and count the fraud detected
for index, row in results_decision_tree.iterrows():
    if row['Fraud_Status'] == 'Fraud':
        count_fraud_decision_tree += 1

# Print the count of fraud detected for Decision Tree
print(f"Count of Fraud Detected for Decision Tree: {count_fraud_decision_tree}")


Count of Fraud Detected for Decision Tree: 2366


In [39]:
#Random Forest Model:
from sklearn.ensemble import RandomForestClassifier

# Create and fit the Random Forest model
random_forest = RandomForestClassifier(random_state=42, n_estimators=100)
random_forest.fit(x_train, y_train)

# Make predictions
y_pred_rf = random_forest.predict(x_test)

# Calculate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy for Random Forest:", accuracy_rf)

# Display the classification report
print("\nClassification Report for Random Forest:\n", classification_report(y_test, y_pred_rf))


Accuracy for Random Forest: 0.9979899913445465

Classification Report for Random Forest:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.82      0.61      0.70      2145

    accuracy                           1.00    555719
   macro avg       0.91      0.81      0.85    555719
weighted avg       1.00      1.00      1.00    555719



In [47]:
# Create a new DataFrame to store the results for Random Forest
results_random_forest = pd.DataFrame({
    'Card_Identifier': test_df['cc_num'],  # Assuming 'cc_num' is the card identifier
    'Random_Forest_Detection': y_pred_rf,
    'Fraud_Status': ['Fraud' if detection == 1 else 'Legitimate' for detection in y_pred_rf]
})


# Initialize a count for fraud detected
count_fraud_random_forest = 0

# Iterate through the results for Random Forest and count the fraud detected
for index, row in results_random_forest.iterrows():
    if row['Fraud_Status'] == 'Fraud':
        count_fraud_random_forest += 1

# Print the count of fraud detected for Random Forest
print(f"Count of Fraud Detected for Random Forest: {count_fraud_random_forest}")


Count of Fraud Detected for Random Forest: 1604
